# netflow.py
Solve a multicommodity flow model using Gurobi Optimizer.


## List data
  0. Commodities (products)
  0. Nodes in the network
  0. Arcs in the network

In [1]:
from gurobipy import *

commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'New York', 'Seattle']
arcs = [
  ('Detroit', 'Boston'), ('Detroit', 'New York'), ('Detroit', 'Seattle'),
  ('Denver',  'Boston'), ('Denver',  'New York'), ('Denver',  'Seattle')]

## Indexed data
  0. Capacity: indexed by arcs
  0. Cost: indexed by commodities and arcs
  0. Inflow: indexed by commodities and nodes

In [4]:
# Maximum transport quantity (of all products together) per route
capacity = {
  ('Detroit', 'Boston'):   100,
  ('Detroit', 'New York'):  80,
  ('Detroit', 'Seattle'):  120,
  ('Denver',  'Boston'):   120,
  ('Denver',  'New York'): 120,
  ('Denver',  'Seattle'):  120 }

# Cost per transported unit of a specific product on a route
cost = {
  ('Pencils', 'Detroit', 'Boston'):   10,  ('Pens', 'Detroit', 'Boston'):   20,
  ('Pencils', 'Detroit', 'New York'): 20,  ('Pens', 'Detroit', 'New York'): 20,
  ('Pencils', 'Detroit', 'Seattle'):  60,  ('Pens', 'Detroit', 'Seattle'):  80,
  ('Pencils', 'Denver',  'Boston'):   40,  ('Pens', 'Denver',  'Boston'):   60,
  ('Pencils', 'Denver',  'New York'): 40,  ('Pens', 'Denver',  'New York'): 70,
  ('Pencils', 'Denver',  'Seattle'):  30,  ('Pens', 'Denver',  'Seattle'):  30 }

# Inflow is which is the amount of flow at each node of the network and that is also indexed by
# a combination of the commedity and the nodes.
# We have 5 nodes and 2 commedities, so we have 10 different inflow values.
# The inflow is the value or the availability of pencils in the node Detroit is 50, 
# and the availabilty of pens in the same node of Detroit is 60.
inflow = {
  ('Pencils', 'Detroit'):   50,  ('Pens', 'Detroit'):   60,
  ('Pencils', 'Denver'):    60,  ('Pens', 'Denver'):    40,
  ('Pencils', 'Boston'):   -50,  ('Pens', 'Boston'):   -40,
  ('Pencils', 'New York'): -50,  ('Pens', 'New York'): -30,
  ('Pencils', 'Seattle'):  -10,  ('Pens', 'Seattle'):  -30 }

In [6]:
for h in commodities:
    for c in nodes:
        print("Inflow " + h + " " + c + ": " + str(inflow[h,c]))

Inflow Pencils Detroit: 50
Inflow Pencils Denver: 60
Inflow Pencils Boston: -50
Inflow Pencils New York: -50
Inflow Pencils Seattle: -10
Inflow Pens Detroit: 60
Inflow Pens Denver: 40
Inflow Pens Boston: -40
Inflow Pens New York: -30
Inflow Pens Seattle: -30


## Create model, decision variables and objective

- Use `Model.addVars()` to add the decision variables
- With two arguments, it takes the cross product of the commodities and the arcs

In [9]:
m = Model("netflow")

flow = m.addVars(commodities, arcs, obj=cost, name="flow")
flow

Set parameter Username
Set parameter LicenseID to value 2588857
Academic license - for non-commercial use only - expires 2025-11-22


{('Pencils', 'Detroit', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Detroit', 'New York'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Detroit', 'Seattle'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Denver', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Denver', 'New York'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Denver', 'Seattle'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Detroit', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Detroit', 'New York'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Detroit', 'Seattle'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Denver', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Denver', 'New York'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Denver', 'Seattle'): <gurobi.Var *Awaiting Model Update*>}

## Create constraints

- Use `Model.addConstrs()` to add the constraints
- Uses two **Python Generator expressions**
    - To generate an arc capacity constraint for every arc _i,j_
    - To generate a flow conservation constraint for every commodity _h_ and every node _j_
- Inside each constraint, uses the aggregate operator `tupledict.sum()` to compute the sum over only the matching elements

In [12]:
# Arc capacities
cap = m.addConstrs(
    (flow.sum('*', i, j) <= capacity[i,j] for i,j in arcs), "cap")

# Flow conservation
node = m.addConstrs(
    (flow.sum(h, '*', j) + inflow[h,j] == flow.sum(h,j,'*')
    for h in commodities for j in nodes), "node")

In [14]:
cap

{('Detroit', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Detroit', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Denver', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [16]:
node

{('Pencils', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'New York'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

## Solve and print the flows

In [19]:
m.optimize()
m.printAttr('X')

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 12 columns and 36 nonzeros
Model fingerprint: 0xc43e5943
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra simplex iterations after uncrush: 1
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.500000000e+03

    Variable            X 
-------------------------
flow[Pencils,Detroit,Bos

## Output
Display the solution as a chart and as a table

In [22]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.DataFrame({
    'arcs': ["%s-%s" % (i,j) for h,i,j in keys],
    'commodities': [h for h,i,j in keys],
    'flow': [flow[h,i,j].X for h,i,j in keys]
})

pivot_df = df.pivot(index='arcs', columns='commodities', values='flow')
pivot_df.plot(kind='bar', stacked=True)
plt.show()

NameError: name 'keys' is not defined

In [ ]:
from bokeh.charts import *
output_notebook()

keys = sorted(flow.keys())
data = {
    'arcs': ["%s-%s"% (i,j) for h,i,j in keys],
    'commodities': [h for h,i,j in keys],
    'flow': [flow[h,i,j].X for h,i,j in keys],
    'use': [flow[h,i,j].X/capacity[i,j] for h,i,j in keys],
}
bar = Bar(data, values='flow', label='arcs', stack='commodities', title="Network flow")
show(bar)

In [25]:
import pandas as pd
mi = pd.MultiIndex.from_tuples(sorted(list(arcs)), names=('origin','destination'))
df = pd.DataFrame(index=mi, columns=commodities)
for h in commodities:
    for i,j in arcs:
        df[h][i,j] = flow[h,i,j].X

df

C:\Users\bmohammad\AppData\Local\Temp\ipykernel_18204\1750227924.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[h][i,j] = flow[h,i,j].X


Pencils  Pens
origin  destination              
Denver  Boston          0.0  10.0
        New York       50.0   0.0
        Seattle        10.0  30.0
Detroit Boston         50.0  30.0
        New York        0.0  30.0
        Seattle         0.0   0.0

## Debugging
Write the model as an LP file

In [28]:
m.write("netflow.lp")